In [1]:
!C:\Users\Sony\Desktop\автобрея\udpipe-1.2.0-bin\bin-win64\udpipe --input horizontal --output conllu \
--tokenize --tag --parse \
C:\Users\Sony\Desktop\автобрея\russian-syntagrus-ud-2.4-190531.udpipe \
< corpus.txt > corpus.conllu

Loading UDPipe model: done.


In [2]:
from nltk.parse import DependencyGraph

In [3]:
trees = []

with open('corpus.conllu', encoding='utf-8') as f:
    parsed_sents = f.read().split('\n\n')

    for sent in parsed_sents:
        tree = [line for line in sent.split('\n') if line and line[0] != '#']
        trees.append('\n'.join(tree))

In [19]:
len(trees)

6482

In [14]:
verb_count = {}
for t in trees:
    try:
        g = DependencyGraph(t, top_relation_label='root')
    except:
        continue
    for n in g.nodes:
        if g.nodes[n]['ctag'] == 'VERB':
            if g.nodes[n]['lemma'] in verb_count:
                verb_count[g.nodes[n]['lemma']] += 1
            else:
                verb_count[g.nodes[n]['lemma']] = 1

In [18]:
good_verbs = []
for verb in verb_count:
    if verb_count[verb] >= 50:
        print(verb, verb_count[verb])
        good_verbs.append(verb)

заявить 258
подать 274
просить 143
признать 343
стать 213
сообщить 216
обратиться 139
отказаться 117
рассматривать 99
рассказать 54
рассмотреть 90
мочь 349
начаться 74
обвинить 461
решить 86
пытаться 89
использовать 56
напомнить 97
получить 125
передать 68
направить 105
удаться 61
сказать 98
иметь 94
делать 77
обжаловать 102
отказать 59
говорить 139
счесть 55
быть 136
оставить 61
дать 70
удовлетворить 173
вынести 152
требовать 172
являться 113
обвинять 119
принять 151
выплатить 59
назначить 63
постановить 54
обязать 58
принадлежать 54
подтвердить 91
находиться 117
арестовывать 82
доказать 56
пояснить 58
предъявить 97
согласиться 62
взыскать 62
отклонить 69
провести 63
приговорить 122
запретить 55
считать 152
оспаривать 54
нет 68
утверждать 71
пройти 50
связать 56
оспорить 68
объявить 72
оказаться 62
отменить 83


In [26]:
verb_phrase = []
for t in trees:
    try:
        g = DependencyGraph(t, top_relation_label='root')
    except:
        continue
    for n in g.nodes:
        if g.nodes[n]['lemma'] in good_verbs and n < len(g.nodes) - 1 and n != 0:
            if g.nodes[n+1]['feats'] and 'Case=Acc' in g.nodes[n+1]['feats'] and g.nodes[n+1]['ctag'] == 'NOUN':
                verb_phrase.append([g.nodes[n]['lemma'], g.nodes[n+1]['word']])
            elif g.nodes[n-1]['feats'] and 'Case=Acc' in g.nodes[n-1]['feats'] and g.nodes[n-1]['ctag'] == 'NOUN':
                verb_phrase.append([g.nodes[n]['lemma'], g.nodes[n-1]['word']])

In [28]:
verb_phrase[:15]

[['рассмотреть', 'вопрос'],
 ['начаться', 'понедельник'],
 ['получить', 'сроки'],
 ['обвинить', 'олигарха'],
 ['просить', 'суд'],
 ['рассматривать', 'ходатайство'],
 ['иметь', 'вид'],
 ['дать', 'согласие'],
 ['признать', 'сделку'],
 ['обвинить', 'господина'],
 ['удовлетворить', 'жалобу'],
 ['вынести', 'постановление'],
 ['мочь', 'сроки'],
 ['просить', 'суд'],
 ['признать', 'сделку']]

In [29]:
len(verb_phrase)

1346

Оценим полученные словосочетания метриками

In [36]:
import nltk
from nltk.collocations import *

bigram_measures = nltk.collocations.BigramAssocMeasures()
finder = BigramCollocationFinder.from_documents(verb_phrase)

In [37]:
pmi = finder.nbest(bigram_measures.pmi, 100)
pmi[:15]

[('нет', 'соглашение'),
 ('связать', 'подзащитного'),
 ('говорить', 'друга'),
 ('пояснить', 'суть'),
 ('назначить', 'наказание'),
 ('постановить', 'присяжные'),
 ('находиться', 'долг'),
 ('начаться', 'очередь'),
 ('пояснить', 'адвокат'),
 ('принадлежать', 'имя'),
 ('принадлежать', 'танкеры'),
 ('принадлежать', 'экс-премьеру'),
 ('отказать', 'сети'),
 ('объявить', 'голодовку'),
 ('сообщить', 'сведения')]

In [39]:
scores = finder.score_ngrams(bigram_measures.dice)
dice = sorted([x for x in scores if x[1] != 1.0], key=lambda x: x[1], reverse=True)[:100]
dice = [x[0] for x in dice]
dice[:15]

[('пройти', 'прения'),
 ('говорить', 'друга'),
 ('заявить', 'отвод'),
 ('пояснить', 'суть'),
 ('просить', 'суд'),
 ('сообщить', 'ответ'),
 ('делать', 'вывод'),
 ('иметь', 'право'),
 ('назначить', 'наказание'),
 ('объявить', 'перерыв'),
 ('постановить', 'присяжные'),
 ('пояснить', 'адвокат'),
 ('находиться', 'время'),
 ('подать', 'иск'),
 ('доказать', 'невиновность')]

In [38]:
logl = finder.nbest(bigram_measures.likelihood_ratio, 100)
logl[:15]

[('просить', 'суд'),
 ('подать', 'иск'),
 ('принять', 'решение'),
 ('удовлетворить', 'иск'),
 ('обжаловать', 'решение'),
 ('удовлетворить', 'ходатайство'),
 ('вынести', 'решение'),
 ('иметь', 'право'),
 ('пройти', 'прения'),
 ('вынести', 'приговор'),
 ('обвинить', 'компанию'),
 ('подать', 'апелляцию'),
 ('делать', 'вывод'),
 ('признать', 'правоту'),
 ('рассмотреть', 'жалобу')]

In [72]:
## получаем пересечение трех метрик
intersect = set(pmi) & set(dice) & set(logl)
print(len(intersect))
intersect

18


{('арестовывать', 'имущество'),
 ('арестовывать', 'счета'),
 ('взыскать', 'деньги'),
 ('говорить', 'друга'),
 ('делать', 'вывод'),
 ('доказать', 'невиновность'),
 ('запретить', 'деятельность'),
 ('заявить', 'отвод'),
 ('мочь', 'список'),
 ('находиться', 'время'),
 ('объявить', 'перерыв'),
 ('объявить', 'предпринимателя'),
 ('передать', 'приказ'),
 ('провести', 'обыски'),
 ('пройти', 'прения'),
 ('решить', 'проблему'),
 ('сообщить', 'ответ'),
 ('счесть', 'доводы')}

Теперь пересечем со словарем гл. сочетаемости

In [46]:
with open("verb_coll.txt", "r", encoding='utf-8') as d:
    lines = d.readlines()

In [47]:
lines[:10]

['аборт\tbase=действие mood=объект order=NV\tаборт делать\n',
 'аборт\tbase=действие mood=объект order=VN\tделать аборт\n',
 'аборт\tbase=действие mood=объект order=VN\tсделать аборт\n',
 'авария\tbase=существование/создание mood=субъект semclass=факт order=NV\tавария произошла\n',
 'авария\tbase=существование/создание mood=субъект semclass=факт order=NV\tавария случилась\n',
 'авария\tbase=действие mood=объект order=VN\tпотерпеть аварию\n',
 'авария\tbase=существование/создание mood=субъект semclass=факт order=VN\tпроизошла авария\n',
 'авария\tbase=существование/создание mood=субъект semclass=факт order=VN\tслучилась авария\n',
 'авторитет\tbase=обладание phase=начало order=VN\tзавоевать авторитет\n',
 'авторитет\tsemclass=содействие subclass=вред order=VN\tподорвать авторитет\n']

In [73]:
dict_coll = []
for line in lines:
    clean_line = line.strip('\t').strip('\n')
    if len(clean_line.split()) == 6:
        dict_coll.append((clean_line.split()[4], clean_line.split()[5]))
        
dict_coll[:10] ## попадают лишние штуки, ну ладно...

[('аборт', 'делать'),
 ('делать', 'аборт'),
 ('сделать', 'аборт'),
 ('потерпеть', 'аварию'),
 ('завоевать', 'авторитет'),
 ('подорвать', 'авторитет'),
 ('подорвать', 'авторитет'),
 ('половой', 'акт'),
 ('половой', 'акт'),
 ('половой', 'акт')]

In [74]:
dict_coll_rev = []
for line in lines:
    clean_line = line.strip('\t').strip('\n')
    if len(clean_line.split()) == 6:
        dict_coll_rev.append((clean_line.split()[5], clean_line.split()[4]))
        
dict_coll_rev[:10]

[('делать', 'аборт'),
 ('аборт', 'делать'),
 ('аборт', 'сделать'),
 ('аварию', 'потерпеть'),
 ('авторитет', 'завоевать'),
 ('авторитет', 'подорвать'),
 ('авторитет', 'подорвать'),
 ('акт', 'половой'),
 ('акт', 'половой'),
 ('акт', 'половой')]

In [69]:
## получаем пересечение со словарем гл. сочетаемости
new_intersect = intersect & set(dict_coll_rev) | intersect & set(dict_coll)
new_intersect

{('объявить', 'перерыв'), ('решить', 'проблему')}

Как-то уж совсем мало получилось... Но выглядят они нормально. 
Из тех 18-и словосочетаний, я бы добавила к получившимся коллокациям словосочетания: делать вывод, провести обыск(и), передать приказ, доказать невиновность, сообщить ответ. Эти словосочетания являются устойчивыми выражениями (нельзя сказать "готовить вывод" вместо "делать", "сделать обыск" и т.д.)

In [79]:
my_set = set()
string = 'делать вывод, провести обыск, передать приказ, доказать невиновность, сообщить ответ'
for coll in string.split(','):
    my_set.add(tuple(coll.split()))

In [80]:
my_set

{('делать', 'вывод'),
 ('доказать', 'невиновность'),
 ('передать', 'приказ'),
 ('провести', 'обыск'),
 ('сообщить', 'ответ')}

In [81]:
new_intersect = new_intersect | my_set

In [82]:
new_intersect

{('делать', 'вывод'),
 ('доказать', 'невиновность'),
 ('объявить', 'перерыв'),
 ('передать', 'приказ'),
 ('провести', 'обыск'),
 ('решить', 'проблему'),
 ('сообщить', 'ответ')}

In [88]:
## распределила на основе собственного мироощущения, не очень понятно, по каким критериям это нужно было делать
collocations = [['провести', 'обыск'], ['объявить', 'перерыв'], ['передать', 'приказ'], ['сообщить', 'ответ'],
               ['делать', 'вывод'], ['доказать', 'невиновность'], ['решить', 'проблему']]

In [89]:
collocations

[['провести', 'обыск'],
 ['объявить', 'перерыв'],
 ['передать', 'приказ'],
 ['сообщить', 'ответ'],
 ['делать', 'вывод'],
 ['доказать', 'невиновность'],
 ['решить', 'проблему']]

In [90]:
from scipy import stats

stats.spearmanr(collocations)

C:\Users\Sony\Anaconda3\lib\site-packages\scipy\stats\stats.py:253: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)


SpearmanrResult(correlation=0.6071428571428572, pvalue=0.1482311614811614)

In [96]:
finder2 = BigramCollocationFinder.from_documents(collocations)
pmi = finder2.nbest(bigram_measures.pmi, 7)
print(stats.spearmanr(pmi))

SpearmanrResult(correlation=0.6071428571428572, pvalue=0.1482311614811614)


C:\Users\Sony\Anaconda3\lib\site-packages\scipy\stats\stats.py:253: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)


In [97]:
logl = finder2.nbest(bigram_measures.likelihood_ratio, 7)
print(stats.spearmanr(logl))

SpearmanrResult(correlation=0.6071428571428572, pvalue=0.1482311614811614)


C:\Users\Sony\Anaconda3\lib\site-packages\scipy\stats\stats.py:253: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)


In [98]:
scores = finder2.score_ngrams(bigram_measures.dice)
dice = [x[0] for x in scores]
print(stats.spearmanr(dice))

SpearmanrResult(correlation=0.6071428571428572, pvalue=0.1482311614811614)


C:\Users\Sony\Anaconda3\lib\site-packages\scipy\stats\stats.py:253: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)


Все метрики сработали одинаково